In [1]:
import json
import os
import sys
from datetime import datetime

# Attempt to import python-docx, but do not exit if missing
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    Document = None
    Pt = None
    print("Warning: python-docx not found. Report generation will be disabled.")

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Date input helper with auto-dashes and year range validation

def input_date(prompt, min_year, max_year):
    """
    Prompt for a date in YYYYMMDD or YYYY-MM-DD format,
    auto-insert dashes, and validate year within range.
    Returns: date string in 'YYYY-MM-DD'.
    """
    while True:
        raw = input(f"{prompt} (YYYYMMDD): ").strip()
        # Extract digits
        digits = ''.join(filter(str.isdigit, raw))
        if len(digits) != 8:
            print("Please enter the date as 8 digits (YYYYMMDD).")
            continue
        year, month, day = int(digits[:4]), int(digits[4:6]), int(digits[6:8])
        try:
            date_obj = datetime(year, month, day)
        except ValueError:
            print("Invalid date. Please try again.")
            continue
        if year < min_year or year > max_year:
            print(f"Year must be between {min_year} and {max_year}.")
            continue
        return date_obj.strftime("%Y-%m-%d")

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1

    print("--- Add New Patient ---")
    name = input("Enter patient name: ").strip()
    dob = input_date("Enter date of birth", 1920, datetime.now().year)
    gender = input("Enter gender: ").strip()

    patient = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'gender': gender
    }
    patients.append(patient)
    _save_data(PATIENTS_FILE, patients)
    print(f"Patient '{name}' added with ID {new_id}.")


def list_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print("No patients found.")
        return
    print("--- Patient List ---")
    for p in patients:
        print(f"ID: {p['id']} | Name: {p['name']} | DOB: {p['dob']} | Gender: {p['gender']}")


def search_patient():
    term = input("Enter name to search: ").strip().lower()
    patients = _load_data(PATIENTS_FILE)
    matches = [p for p in patients if term in p['name'].lower()]
    if not matches:
        print(f"No patients found matching '{term}'.")
        return
    print(f"--- Search Results for '{term}' ---")
    for p in matches:
        print(f"ID: {p['id']} | Name: {p['name']} | DOB: {p['dob']} | Gender: {p['gender']}")

# Notes management

def add_note():
    notes = _load_data(NOTES_FILE)
    new_id = len(notes) + 1

    print("--- Add Clinical Note ---")
    patient_id = input("Enter patient ID: ").strip()
    session_date = input_date("Enter session date", 2010, datetime.now().year)
    subjective = input("Subjective: ").strip()
    objective = input("Objective: ").strip()
    assessment = input("Assessment: ").strip()
    plan = input("Plan: ").strip()

    note = {
        'id': new_id,
        'patient_id': patient_id,
        'session_date': session_date,
        'subjective': subjective,
        'objective': objective,
        'assessment': assessment,
        'plan': plan
    }
    notes.append(note)
    _save_data(NOTES_FILE, notes)
    print(f"Clinical note added with ID {new_id}.")


def list_notes():
    notes = _load_data(NOTES_FILE)
    if not notes:
        print("No clinical notes found.")
        return
    print("--- Clinical Notes List ---")
    for n in notes:
        print(f"ID: {n['id']} | Patient ID: {n['patient_id']} | Date: {n['session_date']}")


def generate_report():
    patients = _load_data(PATIENTS_FILE)
    pid = input("Enter patient ID for report: ").strip()
    patient = next((p for p in patients if str(p['id']) == pid), None)
    if not patient:
        print(f"No patient with ID {pid}.")
        return
    notes = _load_data(NOTES_FILE)
    patient_notes = [n for n in notes if str(n['patient_id']) == pid]
    if Document is None:
        print("python-docx not available. Cannot generate report.")
        return
    doc = Document()
    doc.add_heading(f"Patient Report: {patient['name']} (ID: {pid})", level=1)
    doc.add_paragraph(f"DOB: {patient['dob']}")
    doc.add_paragraph(f"Gender: {patient['gender']}")
    for n in patient_notes:
        doc.add_heading(f"Session: {n['session_date']}", level=2)
        doc.add_paragraph(f"Subjective: {n['subjective']}")
        doc.add_paragraph(f"Objective: {n['objective']}")
        doc.add_paragraph(f"Assessment: {n['assessment']}")
        doc.add_paragraph(f"Plan: {n['plan']}")
    filename = f"{patient['name'].replace(' ', '_')}_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"
    doc.save(filename)
    print(f"Report saved to {filename}")

# Main menu

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('List patients', list_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add clinical note', add_note),
        '5': ('List clinical notes', list_notes),
        '6': ('Generate report', generate_report)
    }
    while True:
        print("\n=== Physio EHR CLI ===")
        for key, (desc, _) in options.items():
            print(f"{key}. {desc}")
        print("0. Exit")
        choice = input("Select an option: ").strip()
        if choice == '0':
            print("Exiting program.")
            break
        action = options.get(choice)
        if action:
            action[1]()
        else:
            print("Invalid selection. Please try again.")

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


Select an option:  0


Exiting program.


In [3]:
import json
import os
import sys
from datetime import datetime

# Attempt to import python-docx, but do not exit if missing
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    Document = None
    Pt = None
    print("Warning: python-docx not found. Report generation will be disabled.")

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Date input helper with auto-dashes and year range validation

def input_date(prompt, min_year, max_year):
    """
    Prompt for a date in YYYYMMDD or YYYY-MM-DD format,
    auto-insert dashes, and validate year within range.
    Returns: date string in 'YYYY-MM-DD'.
    """
    while True:
        raw = input(f"{prompt} (YYYYMMDD): ").strip()
        # Strip non-digits, expect exactly 8 digits
        digits = ''.join(filter(str.isdigit, raw))
        if len(digits) != 8:
            print("Please enter exactly 8 digits (YYYYMMDD).")
            continue

        year, month, day = int(digits[:4]), int(digits[4:6]), int(digits[6:8])
        try:
            dt = datetime(year, month, day)
        except ValueError:
            print("That date is invalid. Please try again.")
            continue

        if year < min_year or year > max_year:
            print(f"Year must be between {min_year} and {max_year}.")
            continue

        return dt.strftime("%Y-%m-%d")

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1

    print("\n--- Add New Patient ---")
    name = input("Enter patient name: ").strip()
    dob  = input_date("Enter date of birth", 1920, datetime.now().year)
    gender = input("Enter gender: ").strip()

    patient = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'gender': gender
    }
    patients.append(patient)
    _save_data(PATIENTS_FILE, patients)
    print(f"Patient '{name}' added with ID {new_id}.")

def list_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print("No patients found.")
        return
    print("\n--- Patient List ---")
    for p in patients:
        print(f"ID: {p['id']} | Name: {p['name']} | DOB: {p['dob']} | Gender: {p['gender']}")

def search_patient():
    term = input("Enter name to search: ").strip().lower()
    patients = _load_data(PATIENTS_FILE)
    matches = [p for p in patients if term in p['name'].lower()]
    if not matches:
        print(f"No patients found matching '{term}'.")
        return
    print(f"\n--- Search Results for '{term}' ---")
    for p in matches:
        print(f"ID: {p['id']} | Name: {p['name']} | DOB: {p['dob']} | Gender: {p['gender']}")

# Clinical notes management

def add_note():
    notes = _load_data(NOTES_FILE)
    new_id = len(notes) + 1

    print("\n--- Add Clinical Note ---")
    pid = input("Enter patient ID: ").strip()
    session_date = input_date("Enter session date", 2010, datetime.now().year)
    subjective = input("Subjective: ").strip()
    objective  = input("Objective: ").strip()
    assessment = input("Assessment: ").strip()
    plan       = input("Plan: ").strip()

    note = {
        'id': new_id,
        'patient_id': pid,
        'session_date': session_date,
        'subjective': subjective,
        'objective': objective,
        'assessment': assessment,
        'plan': plan
    }
    notes.append(note)
    _save_data(NOTES_FILE, notes)
    print(f"Clinical note added with ID {new_id}.")

def list_notes():
    notes = _load_data(NOTES_FILE)
    if not notes:
        print("No clinical notes found.")
        return
    print("\n--- Clinical Notes List ---")
    for n in notes:
        print(f"ID: {n['id']} | Patient ID: {n['patient_id']} | Date: {n['session_date']}")

def generate_report():
    patients = _load_data(PATIENTS_FILE)
    pid = input("Enter patient ID for report: ").strip()
    patient = next((p for p in patients if str(p['id']) == pid), None)
    if not patient:
        print(f"No patient with ID {pid}.")
        return

    notes = _load_data(NOTES_FILE)
    patient_notes = [n for n in notes if str(n['patient_id']) == pid]

    if Document is None:
        print("python-docx not available. Cannot generate report.")
        return

    doc = Document()
    doc.add_heading(f"Patient Report: {patient['name']} (ID: {pid})", level=1)
    doc.add_paragraph(f"DOB: {patient['dob']}")
    doc.add_paragraph(f"Gender: {patient['gender']}")

    for n in patient_notes:
        doc.add_heading(f"Session: {n['session_date']}", level=2)
        doc.add_paragraph(f"Subjective: {n['subjective']}")
        doc.add_paragraph(f"Objective: {n['objective']}")
        doc.add_paragraph(f"Assessment: {n['assessment']}")
        doc.add_paragraph(f"Plan: {n['plan']}")

    fname = f"{patient['name'].replace(' ', '_')}_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"
    doc.save(fname)
    print(f"Report saved to {fname}")

# Main menu

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('List patients', list_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add clinical note', add_note),
        '5': ('List clinical notes', list_notes),
        '6': ('Generate report', generate_report)
    }

    while True:
        print("\n=== Physio EHR CLI ===")
        for key, (desc, _) in options.items():
            print(f"{key}. {desc}")
        print("0. Exit")

        choice = input("Select an option: ").strip()
        if choice == '0':
            print("Exiting program.")
            sys.exit(0)

        action = options.get(choice)
        if action:
            action[1]()
        else:
            print("Invalid selection. Please try again.")

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


Select an option:  3
Enter name to search:  Adam



--- Search Results for 'adam' ---
ID: 5 | Name: Adam Stone | DOB: 1999-09-21 | Gender: Male

=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


Select an option:  1



--- Add New Patient ---


...

Patient 'Ricky M' added with ID 6.

=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


Select an option:  0


Exiting program.


SystemExit: 0

C:\Users\andil\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
import json
import os
import sys
from datetime import datetime

# Attempt to import python-docx, but do not exit if missing
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    Document = None
    Pt = None
    print("Warning: python-docx not found. Report generation will be disabled.")

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Date input helper with auto-dashes and year range validation

def input_date(prompt, min_year, max_year):
    """
    Prompt for a date in YYYYMMDD or YYYY-MM-DD format,
    auto-insert dashes, and validate year within range.
    Returns: date string in 'YYYY-MM-DD'.
    """
    while True:
        raw = input(f"{prompt} (YYYYMMDD): ").strip()
        digits = ''.join(filter(str.isdigit, raw))
        if len(digits) != 8:
            print("Please enter exactly 8 digits (YYYYMMDD).")
            continue
        year, month, day = int(digits[:4]), int(digits[4:6]), int(digits[6:8])
        try:
            dt = datetime(year, month, day)
        except ValueError:
            print("That date is invalid. Please try again.")
            continue
        if year < min_year or year > max_year:
            print(f"Year must be between {min_year} and {max_year}.")
            continue
        return dt.strftime("%Y-%m-%d")

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1

    print("\n--- Add New Patient ---")
    name = input("Enter patient name: ").strip()
    dob = input_date("Enter date of birth", 1920, datetime.now().year)
    gender = input("Enter gender: ").strip()
    medical_aid = input("Medical Aid: ").strip()
    medical_aid_number = input("Medical Aid Number: ").strip()
    dependent_code = input("Dependent Code: ").strip()
    contact_cell = input("Contact Cellphone Number: ").strip()
    contact_email = input("Contact Email Address: ").strip()

    patient = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'gender': gender,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'dependent_code': dependent_code,
        'contact_cell': contact_cell,
        'contact_email': contact_email
    }
    patients.append(patient)
    _save_data(PATIENTS_FILE, patients)
    print(f"Patient '{name}' added with ID {new_id}.")


def list_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print("No patients found.")
        return
    print("\n--- Patient List ---")
    for p in patients:
        print(
            f"ID: {p['id']} | Name: {p['name']} | DOB: {p['dob']} | Gender: {p['gender']} |"
            f" Medical Aid: {p['medical_aid']} | MedAid No.: {p['medical_aid_number']} | Dep Code: {p['dependent_code']} |"
            f" Cell: {p['contact_cell']} | Email: {p['contact_email']}"
        )


def search_patient():
    term = input("Enter name to search: ").strip().lower()
    patients = _load_data(PATIENTS_FILE)
    matches = [p for p in patients if term in p['name'].lower()]
    if not matches:
        print(f"No patients found matching '{term}'.")
        return
    print(f"\n--- Search Results for '{term}' ---")
    for p in matches:
        print(
            f"ID: {p['id']} | Name: {p['name']} | DOB: {p['dob']} | Gender: {p['gender']} |"
            f" Medical Aid: {p['medical_aid']} | MedAid No.: {p['medical_aid_number']} | Dep Code: {p['dependent_code']} |"
            f" Cell: {p['contact_cell']} | Email: {p['contact_email']}"
        )

# Clinical notes management

def add_note():
    notes = _load_data(NOTES_FILE)
    new_id = len(notes) + 1

    print("\n--- Add Clinical Note ---")
    pid = input("Enter patient ID: ").strip()
    session_date = input_date("Enter session date", 2010, datetime.now().year)
    subjective = input("Subjective: ").strip()
    objective = input("Objective: ").strip()
    assessment = input("Assessment: ").strip()
    plan = input("Plan: ").strip()

    note = {
        'id': new_id,
        'patient_id': pid,
        'session_date': session_date,
        'subjective': subjective,
        'objective': objective,
        'assessment': assessment,
        'plan': plan
    }
    notes.append(note)
    _save_data(NOTES_FILE, notes)
    print(f"Clinical note added with ID {new_id}.")


def list_notes():
    notes = _load_data(NOTES_FILE)
    if not notes:
        print("No clinical notes found.")
        return
    print("\n--- Clinical Notes List ---")
    for n in notes:
        print(f"ID: {n['id']} | Patient ID: {n['patient_id']} | Date: {n['session_date']}")


def generate_report():
    patients = _load_data(PATIENTS_FILE)
    pid = input("Enter patient ID for report: ").strip()
    patient = next((p for p in patients if str(p['id']) == pid), None)
    if not patient:
        print(f"No patient with ID {pid}.")
        return

    notes = _load_data(NOTES_FILE)
    patient_notes = [n for n in notes if str(n['patient_id']) == pid]

    if Document is None:
        print("python-docx not available. Cannot generate report.")
        return

    doc = Document()
    doc.add_heading(f"Patient Report: {patient['name']} (ID: {pid})", level=1)
    doc.add_paragraph(f"DOB: {patient['dob']}")
    doc.add_paragraph(f"Gender: {patient['gender']}")
    doc.add_paragraph(f"Medical Aid: {patient['medical_aid']}")
    doc.add_paragraph(f"MedAid Number: {patient['medical_aid_number']}")
    doc.add_paragraph(f"Dependent Code: {patient['dependent_code']}")
    doc.add_paragraph(f"Contact Cell: {patient['contact_cell']}")
    doc.add_paragraph(f"Contact Email: {patient['contact_email']}")

    for n in patient_notes:
        doc.add_heading(f"Session: {n['session_date']}", level=2)
        doc.add_paragraph(f"Subjective: {n['subjective']}")
        doc.add_paragraph(f"Objective: {n['objective']}")
        doc.add_paragraph(f"Assessment: {n['assessment']}")
        doc.add_paragraph(f"Plan: {n['plan']}" )

    fname = f"{patient['name'].replace(' ', '_')}_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"
    doc.save(fname)
    print(f"Report saved to {fname}")

# Main menu

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('List patients', list_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add clinical note', add_note),
        '5': ('List clinical notes', list_notes),
        '6': ('Generate report', generate_report)
    }

    while True:
        print("\n=== Physio EHR CLI ===")
        for key, (desc, _) in options.items():
            print(f"{key}. {desc}")
        print("0. Exit")

        choice = input("Select an option: ").strip()
        if choice == '0':
            print("Exiting program.")
            sys.exit(0)

        action = options.get(choice)
        if action:
            action[1]()
        else:
            print("Invalid selection. Please try again.")

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


Select an option:  1



--- Add New Patient ---


Enter patient name:  Mike
Enter date of birth (YYYYMMDD):  19770209
Enter gender:  Male
Medical Aid:  NMC
Medical Aid Number:  2081254232
Dependent Code:  02
Contact Cellphone Number:  0814225732
Contact Email Address:  ansd@gmail.com


Patient 'Mike' added with ID 7.

=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


Select an option:  2



--- Patient List ---


KeyError: 'gender'

In [ ]:
import json
import os
import sys
from datetime import datetime

# Attempt to import python-docx, but do not exit if missing
try:
    from docx import Document
    from docx.shared import Pt
except ImportError:
    Document = None
    Pt = None
    print("Warning: python-docx not found. Report generation will be disabled.")

PATIENTS_FILE = 'patients.json'
NOTES_FILE = 'notes.json'

# Utility functions

def _load_data(path):
    if not os.path.exists(path):
        return []
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def _save_data(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)

# Date input helper with auto-dashes and year range validation

def input_date(prompt, min_year, max_year):
    """
    Prompt for a date in YYYYMMDD or YYYY-MM-DD format,
    auto-insert dashes, and validate year within range.
    Returns: date string in 'YYYY-MM-DD'.
    """
    while True:
        raw = input(f"{prompt} (YYYYMMDD): ").strip()
        digits = ''.join(filter(str.isdigit, raw))
        if len(digits) != 8:
            print("Please enter exactly 8 digits (YYYYMMDD).")
            continue
        year, month, day = int(digits[:4]), int(digits[4:6]), int(digits[6:8])
        try:
            dt = datetime(year, month, day)
        except ValueError:
            print("That date is invalid. Please try again.")
            continue
        if year < min_year or year > max_year:
            print(f"Year must be between {min_year} and {max_year}.")
            continue
        return dt.strftime("%Y-%m-%d")

# Patient management

def add_patient():
    patients = _load_data(PATIENTS_FILE)
    new_id = len(patients) + 1

    print("\n--- Add New Patient ---")
    name = input("Enter patient name: ").strip() or 'unavailable'
    dob = input_date("Enter date of birth", 1920, datetime.now().year)
    gender = input("Enter gender: ").strip() or 'unavailable'
    medical_aid = input("Medical Aid: ").strip() or 'unavailable'
    medical_aid_number = input("Medical Aid Number: ").strip() or 'unavailable'
    dependent_code = input("Dependent Code: ").strip() or 'unavailable'
    contact_cell = input("Contact Cellphone Number: ").strip() or 'unavailable'
    contact_email = input("Contact Email Address: ").strip() or 'unavailable'

    patient = {
        'id': new_id,
        'name': name,
        'dob': dob,
        'gender': gender,
        'medical_aid': medical_aid,
        'medical_aid_number': medical_aid_number,
        'dependent_code': dependent_code,
        'contact_cell': contact_cell,
        'contact_email': contact_email
    }
    patients.append(patient)
    _save_data(PATIENTS_FILE, patients)
    print(f"Patient '{name}' added with ID {new_id}.")


def list_patients():
    patients = _load_data(PATIENTS_FILE)
    if not patients:
        print("No patients found.")
        return
    print("\n--- Patient List ---")
    for p in patients:
        print(
            f"ID: {p.get('id','unavailable')} | Name: {p.get('name','unavailable')} | DOB: {p.get('dob','unavailable')} | Gender: {p.get('gender','unavailable')} |"
            f" Medical Aid: {p.get('medical_aid','unavailable')} | MedAid No.: {p.get('medical_aid_number','unavailable')} | Dep Code: {p.get('dependent_code','unavailable')} |"
            f" Cell: {p.get('contact_cell','unavailable')} | Email: {p.get('contact_email','unavailable')}"
        )


def search_patient():
    term = input("Enter name to search: ").strip().lower()
    patients = _load_data(PATIENTS_FILE)
    matches = [p for p in patients if term in p.get('name','').lower()]
    if not matches:
        print(f"No patients found matching '{term}'.")
        return
    print(f"\n--- Search Results for '{term}' ---")
    for p in matches:
        print(
            f"ID: {p.get('id','unavailable')} | Name: {p.get('name','unavailable')} | DOB: {p.get('dob','unavailable')} | Gender: {p.get('gender','unavailable')} |"
            f" Medical Aid: {p.get('medical_aid','unavailable')} | MedAid No.: {p.get('medical_aid_number','unavailable')} | Dep Code: {p.get('dependent_code','unavailable')} |"
            f" Cell: {p.get('contact_cell','unavailable')} | Email: {p.get('contact_email','unavailable')}"
        )

# Clinical notes management

def add_note():
    notes = _load_data(NOTES_FILE)
    new_id = len(notes) + 1

    print("\n--- Add Clinical Note ---")
    pid = input("Enter patient ID: ").strip() or 'unavailable'
    session_date = input_date("Enter session date", 2010, datetime.now().year)
    subjective = input("Subjective: ").strip() or 'unavailable'
    objective = input("Objective: ").strip() or 'unavailable'
    assessment = input("Assessment: ").strip() or 'unavailable'
    plan = input("Plan: ").strip() or 'unavailable'

    note = {
        'id': new_id,
        'patient_id': pid,
        'session_date': session_date,
        'subjective': subjective,
        'objective': objective,
        'assessment': assessment,
        'plan': plan
    }
    notes.append(note)
    _save_data(NOTES_FILE, notes)
    print(f"Clinical note added with ID {new_id}.")


def list_notes():
    notes = _load_data(NOTES_FILE)
    if not notes:
        print("No clinical notes found.")
        return
    print("\n--- Clinical Notes List ---")
    for n in notes:
        print(f"ID: {n.get('id','unavailable')} | Patient ID: {n.get('patient_id','unavailable')} | Date: {n.get('session_date','unavailable')}")


def generate_report():
    patients = _load_data(PATIENTS_FILE)
    pid = input("Enter patient ID for report: ").strip()
    patient = next((p for p in patients if str(p.get('id','')) == pid), None)
    if not patient:
        print(f"No patient with ID {pid}.")
        return

    notes = _load_data(NOTES_FILE)
    patient_notes = [n for n in notes if str(n.get('patient_id','')) == pid]

    if Document is None:
        print("python-docx not available. Cannot generate report.")
        return

    doc = Document()
    doc.add_heading(f"Patient Report: {patient.get('name','unavailable')} (ID: {pid})", level=1)
    doc.add_paragraph(f"DOB: {patient.get('dob','unavailable')}")
    doc.add_paragraph(f"Gender: {patient.get('gender','unavailable')}")
    doc.add_paragraph(f"Medical Aid: {patient.get('medical_aid','unavailable')}")
    doc.add_paragraph(f"MedAid Number: {patient.get('medical_aid_number','unavailable')}")
    doc.add_paragraph(f"Dependent Code: {patient.get('dependent_code','unavailable')}")
    doc.add_paragraph(f"Contact Cell: {patient.get('contact_cell','unavailable')}")
    doc.add_paragraph(f"Contact Email: {patient.get('contact_email','unavailable')}")

    for n in patient_notes:
        doc.add_heading(f"Session: {n.get('session_date','unavailable')}", level=2)
        doc.add_paragraph(f"Subjective: {n.get('subjective','unavailable')}")
        doc.add_paragraph(f"Objective: {n.get('objective','unavailable')}")
        doc.add_paragraph(f"Assessment: {n.get('assessment','unavailable')}")
        doc.add_paragraph(f"Plan: {n.get('plan','unavailable')}")

    fname = f"{patient.get('name','unavailable').replace(' ', '_')}_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"
    doc.save(fname)
    print(f"Report saved to {fname}")

# Main menu

def main_menu():
    options = {
        '1': ('Add patient', add_patient),
        '2': ('List patients', list_patients),
        '3': ('Search patients', search_patient),
        '4': ('Add clinical note', add_note),
        '5': ('List clinical notes', list_notes),
        '6': ('Generate report', generate_report)
    }

    while True:
        print("\n=== Physio EHR CLI ===")
        for key, (desc, _) in options.items():
            print(f"{key}. {desc}")
        print("0. Exit")

        choice = input("Select an option: ").strip()
        if choice == '0':
            print("Exiting program.")
            sys.exit(0)

        action = options.get(choice)
        if action:
            action[1]()
        else:
            print("Invalid selection. Please try again.")

if __name__ == '__main__':
    main_menu()



=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


Select an option:  2



--- Patient List ---
ID: 1 | Name: Michael | DOB: 19990-08-31 | Gender: unavailable | Medical Aid: unavailable | MedAid No.: unavailable | Dep Code: unavailable | Cell: unavailable | Email: unavailable
ID: 2 | Name: Tomas | DOB: 2000-11-20 | Gender: unavailable | Medical Aid: unavailable | MedAid No.: unavailable | Dep Code: unavailable | Cell: unavailable | Email: unavailable
ID: 3 | Name: 0 | DOB: we | Gender: unavailable | Medical Aid: unavailable | MedAid No.: unavailable | Dep Code: unavailable | Cell: unavailable | Email: unavailable
ID: 4 | Name: Randif | DOB: 1990-08-21 | Gender: unavailable | Medical Aid: NMC | MedAid No.: 4329857 | Dep Code: unavailable | Cell: unavailable | Email: unavailable
ID: 5 | Name: Adam Stone | DOB: 1999-09-21 | Gender: Male | Medical Aid: unavailable | MedAid No.: unavailable | Dep Code: unavailable | Cell: unavailable | Email: unavailable
ID: 6 | Name: Ricky M | DOB: 1989-07-21 | Gender: Male | Medical Aid: unavailable | MedAid No.: unavailable | 

Select an option:  3
Enter name to search:  Mike



--- Search Results for 'mike' ---
ID: 7 | Name: Mike | DOB: 1977-02-09 | Gender: Male | Medical Aid: NMC | MedAid No.: 2081254232 | Dep Code: 02 | Cell: 0814225732 | Email: ansd@gmail.com

=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


...


--- Add Clinical Note ---


Enter patient ID:  8
Enter session date (YYYYMMDD):  20250422
Subjective:  31-year-old male, left femur fracture, fixed with an IM nail. Pt was involved in a robbery on the 13th of April and sustained gunshot wound at close range. Orif was done on the 20th of April. Patient reports pain in the leg, swelling, and also reports feeling congested in the chest. Patient reports having mobilized already with the hospital walking frame to the toilet. 
Objective:  On assessment, normal breath sounds and normal air entry. He had decreased knee ROM, swelling, and normal ankle ROM. The surgical dressing has been taken off and wounds are dry with no signs of infection. 
Assessment:  We mobilized in the hallway PWB with walking frame (about 40m). We did exercises heel slides, active assisted flexion and bridging. Advised to sit while eating to improve knee flexion. I asked the patient to ask his brother to get crutches organized so we can progress with management.  Did ACBTs as well
Plan:  Continue 

Clinical note added with ID 6.

=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit


Select an option:  6
Enter patient ID for report:  8


Report saved to Jonny_Bar_report_20250427_080434.docx

=== Physio EHR CLI ===
1. Add patient
2. List patients
3. Search patients
4. Add clinical note
5. List clinical notes
6. Generate report
0. Exit
